# Text Summary cnn daily mail

In [86]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
print("tf-version", tf.__version__)

tf-version 2.1.0


In [72]:
MAX_LENGTH = 200

BUFFER_SIZE = 20000

BATCH_SIZE = 16

## Get the dataset

In [73]:
cnn_dailymail = tfds.load(name="cnn_dailymail")

In [74]:
train = cnn_dailymail['train']
test = cnn_dailymail['test']

In [75]:
for item in train.take(1):
    print(item.keys(), "\n")
    print(item['article'], "\n")
    print(item['highlights'])

dict_keys(['article', 'highlights']) 

tf.Tensor(b"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated foo

## Clean Dataset
- Remove the first part of the article 
(By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 .)

- then add a <start> and <stop> Token

In [97]:
def map_func(features):
    article_text = tf.strings.split(features["article"], sep=" . ")[7:]
    sum_text = "<start> "
    for text in article_text:
        sum_text += text
    sum_text += " <stop>"
    article = tf.strings.split(tf.strings.lower(sum_text))
    highlights = tf.strings.split(tf.strings.lower(features['highlights']))
    return article[:MAX_LENGTH], highlights



train_ds = train.map(map_func)
test_ds = test.map(map_func)
for article, highlights in train_ds.take(1):
    print("len article:", len(article))
    print(article, "\n")
    print(highlights)

len article: 175
tf.Tensor(
[b'<start>' b'the' b'bishop' b'of' b'the' b'fargo' b'catholic' b'diocese'
 b'in' b'north' b'dakota' b'has' b'exposed' b'potentially' b'hundreds'
 b'of' b'church' b'members' b'in' b'fargo,' b'grand' b'forks' b'and'
 b'jamestown' b'to' b'the' b'hepatitis' b'a' b'virus' b'in' b'late'
 b'september' b'and' b'early' b'october.' b'the' b'state' b'health'
 b'department' b'has' b'issued' b'an' b'advisory' b'of' b'exposure' b'for'
 b'anyone' b'who' b'attended' b'five' b'churches' b'and' b'took'
 b'communion.' b'bishop' b'john' b'folda' b'(pictured)' b'of' b'the'
 b'fargo' b'catholic' b'diocese' b'in' b'north' b'dakota' b'has'
 b'exposed' b'potentially' b'hundreds' b'of' b'church' b'members' b'in'
 b'fargo,' b'grand' b'forks' b'and' b'jamestown' b'to' b'the' b'hepatitis'
 b'astate' b'immunization' b'program' b'manager' b'molly' b'howell'
 b'says' b'the' b'risk' b'is' b'low,' b'but' b'officials' b'feel' b"it's"
 b'important' b'to' b'alert' b'people' b'to' b'the' b'possi

In [84]:
def get_dataset(ds):
    ds = ds.cache()
    ds = ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    return ds.prefetch(tf.data.experimental.AUTOTUNE)

train_ds = get_dataset(train_ds)
test_ds = get_dataset(test_ds)

# Model
## Use a pretrained Model

https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22